<a href="https://colab.research.google.com/github/RM-RAMASAMY/CMPE-255-Assignments/blob/main/gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install -U pip >/dev/null
!pip install -U setuptools wheel >/dev/null
!pip install autogluon.tabular[all] >/dev/null
!pip install --upgrade mxnet >/dev/null

# About Dataset

## 1. Data Source
This dataset is a synthetic version inspired by the original Credit Risk dataset on Kaggle and enriched with additional variables based on Financial Risk for Loan Approval data. **SMOTENC** was used to simulate new data points to enlarge the instances. The dataset is structured for both categorical and continuous features.

link to the dataset: https://www.kaggle.com/datasets/taweilo/loan-approval-classification-data/data


In [20]:
# prompt: pull dataset from https://www.kaggle.com/datasets/taweilo/loan-approval-classification-data/data

!pip install kaggle >/dev/null

# Download the dataset
!kaggle datasets download -d taweilo/loan-approval-classification-data

# Unzip the dataset
!unzip loan-approval-classification-data.zip

Dataset URL: https://www.kaggle.com/datasets/taweilo/loan-approval-classification-data
License(s): apache-2.0
loan-approval-classification-data.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  loan-approval-classification-data.zip
replace loan_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# Data Usage
The dataset can be used for multiple purposes:

## Exploratory Data Analysis (EDA):
Analyze key features, distribution patterns, and relationships to understand credit risk factors.

## Classification:
Build predictive models to classify the loan_status variable (approved/not approved) for potential applicants.

## Regression:
Develop regression models to predict the credit_score variable based on individual and loan-related attributes.

Mind the data issue from the original data, such as the instance > 100-year-old as age. This dataset provides a rich basis for understanding financial risk factors and simulating predictive modeling processes for loan approval and credit scoring.

In [21]:
# Check GPU availability
import torch
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")


GPU is available


In [22]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
from sklearn.model_selection import train_test_split

# Load a dataset
data = pd.read_csv('loan_data.csv')
display(data)
# Split the data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,27.0,male,Associate,47971.0,6,RENT,15000.0,MEDICAL,15.66,0.31,3.0,645,No,1
44996,37.0,female,Associate,65800.0,17,RENT,9000.0,HOMEIMPROVEMENT,14.07,0.14,11.0,621,No,1
44997,33.0,male,Associate,56942.0,7,RENT,2771.0,DEBTCONSOLIDATION,10.02,0.05,10.0,668,No,1
44998,29.0,male,Bachelor,33164.0,4,RENT,12000.0,EDUCATION,13.23,0.36,6.0,604,No,1


In [27]:
# Train a model using AutoGluon
predictor = TabularPredictor(label='loan_status', eval_metric='accuracy').fit(
    train_data,
    num_gpus=1, #include the GPU
    presets='best_quality', # For higher accuracy
    time_limit=120 # Limit training time (seconds) - adjust as needed
)

No path specified. Models will be saved in: "AutogluonModels/ag-20241218_065502"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.33 GB / 12.67 GB (81.5%)
Disk Space Avail:   78.86 GB / 112.64 GB (70.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be f

In [28]:


# Evaluate the model
results = predictor.evaluate(test_data)
print(results)

# Make predictions on new data (example)
predictions = predictor.predict(test_data.drop(columns=['loan_status']))
print(predictions)

# Save the model
predictor.save()


{'accuracy': 0.9372222222222222, 'balanced_accuracy': 0.8934800247688596, 'mcc': 0.8145028125665135, 'roc_auc': 0.9796862611121787, 'f1': 0.8528262568377182, 'precision': 0.8950246036085292, 'recall': 0.8144278606965174}


TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/AutogluonModels/ag-20241218_065502")


37979    0
9911     0
43386    1
13822    0
44810    1
        ..
14572    0
30384    0
36665    0
30569    0
5321     1
Name: loan_status, Length: 9000, dtype: int64
